In [ ]:
from vmtk import vmtkscripts
from extractionmethods import initialization_image, levelset_segmentation, marching_cubes, image_writer, surface_writer, image_reader, image_cast, vessel_enhancer
import numpy as np
import os
import json
from joblib import Parallel, delayed
from shutil import copyfile

In [ ]:
def run_pre_analysis(dataSetBaseImagePath, datasetDerivedDir, **kwargs):

    image = image_reader(datasetBaseImagePath)
    image_writer(image, os.path.join(datasetDerivedDir, 'input-image.vti'))
    
    casted = image_cast(image)
    image_writer(casted, os.path.join(datasetDerivedDir, 'casted-image.vti'))
    
    enhanced = vessel_enhancer(casted, kwargs['enhanceMethod'], kwargs['enhanceIterations'], kwargs['enhanceDiffusionIterations'])
    image_writer(enhanced, os.path.join(datasetDerivedDir, 'enhanced-image.vti'))
    
    return True
    

In [ ]:
def run_initialize(datasetBaseImagePath, datasetDerivedDir, lowerThreshValue, iterationValue, **kwargs):
    parameterDir = os.path.join(datasetDerivedDir, f'thresh{lowerThreshValue}'.replace('.', '-'), f'iterations{iterationValue}')
    if not os.path.exists(parameterDir):
        os.makedirs(parameterDir)

    copyfile(os.path.join(datasetDerivedDir, 'input-image.vti'),
              os.path.join(parameterDir, 'input-image.vti'))
    
    copyfile(os.path.join(datasetDerivedDir, 'casted-image.vti'),
             os.path.join(parameterDir, 'casted-image.vti'))
             
    copyfile(os.path.join(datasetDerivedDir, 'enhanced-image.vti'),
             os.path.join(parameterDir, 'enhanced-image.vti'))
             
    casted = image_reader(os.path.join(parameterDir, 'casted-image.vti'))

    init = initialization_image(casted, lowerThreshValue, kwargs['upperThreshValue'])
    image_writer(init, os.path.join(parameterDir, 'initialization-image.vti'))
    
    return True

In [ ]:
def run_analysis(datasetBaseImagePath, datasetDerivedDir, lowerThreshValue, iterationValue, **kwargs):
    parameterDir = os.path.join(datasetDerivedDir, f'thresh{lowerThreshValue}'.replace('.', '-'), f'iterations{iterationValue}')
    if not os.path.exists(parameterDir):
        raise NotADirectoryError(parameterDir)
    
    casted = image_reader(os.path.join(parameterDir, 'casted-image.vti'))
    enhanced = image_reader(os.path.join(parameterDir, 'enhanced-image.vti'))
    init = image_reader(os.path.join(parameterDir, 'initialization-image.vti'))

    ls = levelset_segmentation(casted, init, enhanced, 
                               kwargs['propagation'], kwargs['curvature'], kwargs['advection'], 
                               iterationValue)
    image_writer(ls, os.path.join(parameterDir, 'levelset-image.vti'))

    mc = marching_cubes(ls)
    surface_writer(mc, os.path.join(parameterDir, 'surface.vtp'))

    params = {'parameterDir': parameterDir,
              'lowerThreshValue':lowerThreshValue,
              'iterationValue':iterationValue,
              'datasetBaseImagePath':datasetBaseImagePath,
              'processSteps':['read_image', 'initialize_image', 'cast_image', 'vessel_enhancer', 'level_set_evolution', 'marching_cubes']}
    
    for k, v in kwargs.items():
        params[k] = v
    
    with open(os.path.join(parameterDir, 'parmas.json'), 'w+') as f:
        json.dump(params, f)

    return True
        

In [ ]:
lowerThreshValues = [220, 230, 240, 250, 254]
iterationValues = [100, 200, 350]

paramDict = {'upperThreshValue': None,
            'curvature': 1,
            'propagation': 2,
            'advection': 1,
            'enhanceMethod': "ved",
            'enhanceIterations': 3,
            'enhanceDiffusionIterations': 3}

In [ ]:
# [(dataSetDir, inputfilename)]

dataSets = [(os.path.join(os.getcwd(), 'multiphoton-sample', 'multiphoton-sample-1'), 'stack1_2um_404um_cropped.mha'), 
            (os.path.join(os.getcwd(), 'multiphoton-sample', 'multiphoton-sample-2'), 'sstack1_514um_2umstep_08xyspacing_.mha'),
            (os.path.join(os.getcwd(), 'multiphoton-sample', 'multiphoton-sample-3'), 'staxk1.mha')]

for dataSet in dataSets:
    dataSetDir = dataSet[0]
    datasetBaseDir = os.path.join(dataSetDir, 'base')
    datasetBaseImagePath = os.path.join(datasetBaseDir, dataSet[1])

    datasetDerivedDir = os.path.join(dataSetDir, 'derived')
    if not os.path.exists(datasetDerivedDir):
        os.makedirs(datasetDerivedDir)
        print(f'created derived dataset directory at: {datasetDerivedDir}')
        
    run_pre_analysis(datasetBaseImagePath, datasetDerivedDir, **paramDict)
    
    Parallel(n_jobs=12)(delayed(run_initialize)(datasetBaseImagePath, datasetDerivedDir, lowThresh, iteration, **paramDict) for lowThresh in lowerThreshValues for iteration in iterationValues)

    Parallel(n_jobs=15)(delayed(run_analysis)(datasetBaseImagePath, datasetDerivedDir, lowThresh, iteration, **paramDict) for lowThresh in lowerThreshValues for iteration in iterationValues)

In [ ]:
for 